<a href="https://colab.research.google.com/github/AndreSpain2104/AndreSpain2104/blob/main/Lab_Crip2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio Ataque de Padding Oracle en AES-CBC

## Fase 1: Servidor vulnerable AES-CBC con oracle de padding

In [ ]:
!pip install -q pycryptodome

In [ ]:

from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

BLOCK = 16

class VulnerableServer:
    """
    Simula un servicio que:
      - encrypt(m) -> IV || C
      - decrypt(data) -> True/False segun padding PKCS#7. Este
        comportamiento es vulnerable al ataque de padding.
    """
    def __init__(self):
        self.key = get_random_bytes(BLOCK)

    def encrypt(self, plaintext: bytes) -> bytes:
        iv = get_random_bytes(BLOCK)
        padded = pad(plaintext, BLOCK)
        c = AES.new(self.key, AES.MODE_CBC, iv).encrypt(padded)
        return iv + c # IV || C

    def decrypt(self, data: bytes) -> bool:
        """
        Devuelve True si el padding del ultimo bloque descifrado
        es valido;
        False de lo contrario. data debe tener >= 2 bloques y
        longitud multiplo de 16.
        """
        try:
            if len(data) < 2*BLOCK or (len(data) % BLOCK != 0):
                return False
            blocks = [data[i:i+BLOCK] for i in range(0, len(data), BLOCK)]
            iv, cblocks = blocks[0], blocks[1:]
            pt = AES.new(self.key, AES.MODE_CBC, iv).decrypt(b"".join(cblocks))
            unpad(pt, BLOCK) # ValueError si padding incorrecto
            return True
        except ValueError:
            return False


## Fase 2: Diseñando el Ataque a un Solo Bloque (recover_block)

**Objetivo:**
Recuperar un bloque de texto plano $M_i$ dado el bloque cifrado actual `c_curr` y su bloque anterior `c_prev`, usando un *padding oracle* vulnerable. El código implementa la fase 2: atacar un solo bloque (`recover_block`) y por ahora solo recupera el **último byte** del bloque (índice 15 en bloques de 16 bytes).

**Idea central (matemática):**

* El descifrado interno produce un estado intermedio $I = D_k(c_{curr})$.
* El texto plano se obtiene como $M = I \oplus c_{prev}$.
* Para el último byte: $M[15] = I[15] \oplus c_{prev}[15]$. Si conseguimos $I[15]$ podemos obtener $M[15]$.

**Estrategia del ataque (byte final):**

1. Construir una versión modificada de `c_prev` (`c_prev_mod`) y variar únicamente su último byte con todos los valores `g = 0..255`.
2. Para cada `g`, enviar `c_prev_mod || c_curr` al *oracle*.
3. Si el *oracle* responde `True` (padding válido), eso implica que el último byte descifrado con `c_prev_mod` fue `0x01` (PKCS#7).
4. De la igualdad $0x01 = I[15] \oplus g$ se deduce $I[15] = g \oplus 0x01$.
5. Finalmente, recuperar el último byte del texto plano original: `M[15] = I[15] ^ c_prev[15]`.

**Qué hace el código mostrado:**

* Inicializa `intermediate_state` y `recovered_plaintext` como `bytearray(BLOCK)`.
* Fija `padding_value = 1` y `target_byte_index = 15`.
* Crea `c_prev_mod = bytearray(BLOCK)` y prueba `g` de 0 a 255:

  * monta `payload = bytes(c_prev_mod) + c_curr`
  * llama `oracle.decrypt(payload)`
  * si `True`, calcula `I[15] = g ^ 0x01` y `M[15] = I[15] ^ c_prev[15]`, imprime resultados y rompe el bucle.
* Devuelve el bloque parcial (con solo el último byte rellenado).

**Prueba:**

* Se instancia `VulnerableServer()`, cifra un mensaje de 16 bytes, obtiene `iv` y `c1`, y llama `recover_block(server, iv, c1)`.
* Al final imprime el byte recuperado y lo compara con el original.

**Supuestos y notas:**

* Bloque de 16 bytes (BLOCK = 16).
* Padding PKCS#7 (por eso un padding válido para el último byte solo puede ser `0x01`).
* El *oracle* solo indica si el padding es válido (True/False).
* `bytearray` se usa porque es mutable y facilita modificar bytes individuales.

In [ ]:
def recover_block(oracle: VulnerableServer, c_prev: bytes, c_curr: bytes) -> bytes:
    """
    Recupera un bloque de texto plano Mi dado Ci y Ci-1.
    """
    # Inicializamos arrays para guardar nuestros descubrimientos.
    # Usamos bytearray porque es mutable, a diferencia de bytes.
    intermediate_state = bytearray(BLOCK)
    recovered_plaintext = bytearray(BLOCK)

    # ATAQUE AL ÚLTIMO BYTE (p=1, t=15)
    padding_value = 1
    target_byte_index = BLOCK - padding_value # Índice 15

    # Creamos una copia modificable de c_prev.
    # Llenémosla con ceros por ahora para que sea simple.
    c_prev_mod = bytearray(BLOCK)

    print(f"[*] Atacando el byte {target_byte_index}...")

    # Bucle de barrido para encontrar el valor 'g' correcto.
    for g in range(256):
        c_prev_mod[target_byte_index] = g

        # Construimos el payload: IV' || Ci
        payload = bytes(c_prev_mod) + c_curr

        # Consultamos al oráculo
        if oracle.decrypt(payload):
            print(f"[+] ¡Oráculo devolvió True para g = {hex(g)}!")

            # Si el oráculo es True, hemos encontrado nuestro 'g'.
            # Ahora, calculamos el byte del estado intermedio.
            intermediate_state[target_byte_index] = g ^ padding_value

            # Y con él, calculamos el byte del texto plano original.
            recovered_plaintext[target_byte_index] = intermediate_state[target_byte_index] ^ c_prev[target_byte_index]

            print(f"    - I[15] = {hex(intermediate_state[target_byte_index])}")
            print(f"    - M[15] = {hex(recovered_plaintext[target_byte_index])} (carácter: {chr(recovered_plaintext[target_byte_index])})")

            # Rompemos el bucle porque ya encontramos el byte.
            break

    # Por ahora, la función no devolverá el bloque completo.
    return bytes(recovered_plaintext)


# Código de prueba para esta fase
if __name__ == '__main__':
    server = VulnerableServer()

    # Mensaje de prueba (un bloque completo para simplificar)
    # "Este es un test" en español tiene 16 bytes.
    original_message = b'Este es un test.'

    print(f"[*] Mensaje original: {original_message}")

    # Ciframos el mensaje. El servidor nos da IV || C1
    ciphertext = server.encrypt(original_message)
    iv = ciphertext[:BLOCK]
    c1 = ciphertext[BLOCK:]

    print("[*] Ejecutando el ataque para recuperar el primer bloque...")

    recovered = recover_block(server, iv, c1)

    print(f"\n[*] Texto plano recuperado (parcial): {recovered}")
    print(f"[*] Byte 15 recuperado: {chr(recovered[15])}, Original: {chr(original_message[15])}")

[*] Mensaje original: b'Este es un test.'
[*] Ejecutando el ataque para recuperar el primer bloque...
[*] Atacando el byte 15...
[+] ¡Oráculo devolvió True para g = 0x0!
    - I[15] = 0x1
    - M[15] = 0xd2 (carácter: Ò)

[*] Texto plano recuperado (parcial): b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xd2'
[*] Byte 15 recuperado: Ò, Original: .


# Fase 3: Generalizando el Ataque para un Bloque Completo

## Idea central
Para un bloque `c_curr`, el descifrado interno produce `I = D_k(c_curr)` y el texto plano es `M = I ⊕ c_prev`. Si conocemos `I` podemos calcular `M`. Recuperamos `I` **byte a byte** empezando por el último (`BLOCK-1`) hacia la izquierda.

## Forzar padding
En la iteración `p` queremos que el bloque descifrado termine con `p` bytes iguales a `p` (PKCS#7): `... p p p`.  
Para ello fijamos, para cada byte ya descubierto `j > t`,  de modo que esos bytes produzcan `p` al descifrar.

## Buscar el byte activo
Con el sufijo preparado variamos `c_prev_mod[t] = g` (0..255). Si el *oracle* devuelve `True`, el plaintext modificado termina en padding válido `p`, por tanto `M'[t] = p`.  
De `p = I[t] ⊕ g` se obtiene `I[t] = g ⊕ p` y luego `M[t] = I[t] ⊕ c_prev[t]`.

## Por qué funciona
El *oracle* solo indica si el padding es válido. Al manipular `c_prev_mod` explotamos la relación XOR `M = I ⊕ c_prev` para forzar salidas conocidas y deducir `I`.




In [ ]:
# Reemplazamos la función recover_block con la versión completa

def recover_block(oracle: VulnerableServer, c_prev: bytes, c_curr: bytes) -> bytes:
    """
    Recupera un bloque de texto plano Mi dado Ci y Ci-1.
    """
    intermediate_state = bytearray(BLOCK)
    recovered_plaintext = bytearray(BLOCK)
    c_prev_mod = bytearray(BLOCK) # La usaremos para todo el proceso

    # Bucle principal: p va de 1 a 16.
    for p in range(1, BLOCK + 1):
        target_byte_index = BLOCK - p
        print(f"[*] Atacando el byte {target_byte_index} (padding {p})...")

        # 1. Preparar el sufijo (bytes que ya hemos descubierto)
        # Para los bytes j > target_byte_index, los forzamos a valer 'p'.
        for j in range(target_byte_index + 1, BLOCK):
            c_prev_mod[j] = intermediate_state[j] ^ p

        # 2. Barrido del byte activo para encontrar 'g'
        found_g = False
        for g in range(256):
            c_prev_mod[target_byte_index] = g
            payload = bytes(c_prev_mod) + c_curr

            if oracle.decrypt(payload):
                # ¡Lo encontramos!
                print(f"[+] Oráculo devolvió True para g = {hex(g)}")

                # 3. Deducir el estado intermedio y el texto plano
                intermediate_state[target_byte_index] = g ^ p
                recovered_plaintext[target_byte_index] = intermediate_state[target_byte_index] ^ c_prev[target_byte_index]

                print(f"    - I[{target_byte_index}] = {hex(intermediate_state[target_byte_index])}")
                print(f"    - M[{target_byte_index}] = {hex(recovered_plaintext[target_byte_index])} (carácter: {repr(chr(recovered_plaintext[target_byte_index]))})")

                found_g = True
                break

        if not found_g:
            # Esto no debería ocurrir si el oráculo es consistente.
            raise RuntimeError(f"No se pudo encontrar el valor para el byte {target_byte_index}")

    return bytes(recovered_plaintext)

# Código de prueba para esta fase
if __name__ == '__main__':
    server = VulnerableServer()

    # Mensaje de prueba (un bloque completo para simplificar)
    # "Este es un test" en español tiene 16 bytes.
    original_message = b'Este es un test.'

    print(f"[*] Mensaje original: {original_message}")

    # Ciframos el mensaje. El servidor nos da IV || C1
    ciphertext = server.encrypt(original_message)
    iv = ciphertext[:BLOCK]
    c1 = ciphertext[BLOCK:]

    print("[*] Ejecutando el ataque para recuperar el primer bloque...")

    recovered = recover_block(server, iv, c1)

    print(f"\n[*] Texto plano recuperado (parcial): {recovered}")
    print(f"[*] Byte 15 recuperado: {chr(recovered[15])}, Original: {chr(original_message[15])}")

[*] Mensaje original: b'Este es un test.'
[*] Ejecutando el ataque para recuperar el primer bloque...
[*] Atacando el byte 15 (padding 1)...
[+] Oráculo devolvió True para g = 0x0
    - I[15] = 0x1
    - M[15] = 0x44 (carácter: 'D')
[*] Atacando el byte 14 (padding 2)...
[+] Oráculo devolvió True para g = 0x0
    - I[14] = 0x2
    - M[14] = 0x71 (carácter: 'q')
[*] Atacando el byte 13 (padding 3)...
[+] Oráculo devolvió True para g = 0x0
    - I[13] = 0x3
    - M[13] = 0x86 (carácter: '\x86')
[*] Atacando el byte 12 (padding 4)...
[+] Oráculo devolvió True para g = 0x0
    - I[12] = 0x4
    - M[12] = 0xf9 (carácter: 'ù')
[*] Atacando el byte 11 (padding 5)...
[+] Oráculo devolvió True para g = 0x0
    - I[11] = 0x5
    - M[11] = 0xe9 (carácter: 'é')
[*] Atacando el byte 10 (padding 6)...
[+] Oráculo devolvió True para g = 0x0
    - I[10] = 0x6
    - M[10] = 0x5b (carácter: '[')
[*] Atacando el byte 9 (padding 7)...
[+] Oráculo devolvió True para g = 0x0
    - I[9] = 0x7
    - M[9] = 0x

# Fase 4: Orquestando el Ataque Completo (recover_message)

**El Objetivo:** Dada una `oracle` y un `ciphertext` completo (`IV || C1 || C2 || ... || Cl`), nuestra nueva función `recover_message` debe devolver el mensaje original completo y **sin el padding**.

**La Estrategia:** Es sorprendentemente sencilla, ya que todo el trabajo pesado lo hace nuestra función `recover_block`.

1.  **Trocear el Cifrado:** Primero, necesitamos dividir el `ciphertext` en sus bloques constituyentes de 16 bytes. El primer bloque es el `IV`, el segundo es `C1`, el tercero `C2`, y así sucesivamente.

2.  **Iterar y Descifrar:** Crearemos un bucle que recorra los bloques cifrados. En cada iteración `i`, llamaremos a nuestra función `recover_block` con los bloques correctos:
    *   El bloque `i` será el `c_curr`.
    *   El bloque `i-1` será el `c_prev`.

3.  **Concatenar los Resultados:** Iremos guardando cada bloque de texto plano recuperado (`M1`, `M2`, `M3`...) en una lista o búfer.

4.  **Limpiar el Padding:** Una vez que hayamos recuperado todos los bloques y los hayamos unido, el resultado será el texto plano original, pero probablemente con el padding PKCS#7 al final. El último paso es usar una función de `unpad` para eliminar ese relleno y obtener el mensaje original limpio. La propia librería `pycryptodome` nos proporciona una.

In [ ]:
def recover_message(oracle: VulnerableServer, ciphertext: bytes) -> bytes:
    """
    Recupera el mensaje completo aplicando recover_block a cada bloque.
    """
    # 1. Trocear el ciphertext en bloques de 16 bytes.
    blocks = [ciphertext[i:i+BLOCK] for i in range(0, len(ciphertext), BLOCK)]

    # El primer bloque es el IV, el resto son los bloques de ciphertext (C1, C2, ...)
    # blocks = [IV, C1, C2, ...]

    full_plaintext = b''

    # 2. Iterar sobre los bloques cifrados.
    # El primer bloque a descifrar es C1 (índice 1), usando IV (índice 0) como c_prev.
    for i in range(1, len(blocks)):
        c_prev = blocks[i-1]
        c_curr = blocks[i]

        print(f"\n[+] Recuperando bloque {i} del mensaje...")
        recovered_block_plaintext = recover_block(oracle, c_prev, c_curr)
        full_plaintext += recovered_block_plaintext
        print(f"    -> Texto plano del bloque {i} recuperado: {recovered_block_plaintext}")

    # 3. Eliminar el padding del resultado final.
    try:
        unpadded_message = unpad(full_plaintext, BLOCK)
        return unpadded_message
    except ValueError as e:
        print(f"[!] Error al quitar el padding: {e}")
        print("[!] Es posible que el último bloque recuperado no fuera correcto.")
        return full_plaintext

# --- Reemplaza tu bloque __main__ con este para probar todo el flujo ---
if __name__ == '__main__':
    server = VulnerableServer()

    # Mensaje de prueba más largo (más de un bloque)
    original_message = b"Este es un mensaje secreto que es un poco mas largo."

    print(f"[*] Mensaje original a atacar:\n    {original_message}\n")

    # El servidor cifra el mensaje por nosotros
    ciphertext = server.encrypt(original_message)

    print("[*] Servidor generó un ciphertext (IV || C1 || C2 || ...)")
    print(f"[*] Longitud total del ciphertext: {len(ciphertext)} bytes")

    print("\n" + "="*50)
    print("        INICIANDO ATAQUE DE PADDING ORACLE")
    print("="*50)

    # Llamamos a nuestra función principal de ataque
    recovered_message = recover_message(server, ciphertext)

    print("\n" + "="*50)
    print("              ATAQUE FINALIZADO")
    print("="*50)

    print(f"\n[*] Mensaje recuperado final (sin padding):\n    {recovered_message}")

    # Verificación final
    if recovered_message == original_message:
        print("\n[SUCCESS] ¡El mensaje recuperado coincide con el original!")
    else:
        print("\n[FAILURE] El mensaje recuperado NO coincide con el original.")

[*] Mensaje original a atacar:
    b'Este es un mensaje secreto que es un poco mas largo.'

[*] Servidor generó un ciphertext (IV || C1 || C2 || ...)
[*] Longitud total del ciphertext: 80 bytes

        INICIANDO ATAQUE DE PADDING ORACLE

[+] Recuperando bloque 1 del mensaje...
[*] Atacando el byte 15 (padding 1)...
[+] Oráculo devolvió True para g = 0xb4
    - I[15] = 0xb5
    - M[15] = 0x61 (carácter: 'a')
[*] Atacando el byte 14 (padding 2)...
[+] Oráculo devolvió True para g = 0x66
    - I[14] = 0x64
    - M[14] = 0x73 (carácter: 's')
[*] Atacando el byte 13 (padding 3)...
[+] Oráculo devolvió True para g = 0x14
    - I[13] = 0x17
    - M[13] = 0x6e (carácter: 'n')
[*] Atacando el byte 12 (padding 4)...
[+] Oráculo devolvió True para g = 0x83
    - I[12] = 0x87
    - M[12] = 0x65 (carácter: 'e')
[*] Atacando el byte 11 (padding 5)...
[+] Oráculo devolvió True para g = 0x6b
    - I[11] = 0x6e
    - M[11] = 0x6d (carácter: 'm')
[*] Atacando el byte 10 (padding 6)...
[+] Oráculo devol